In [22]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import random_split, TensorDataset

import datetime
import os

import optuna
import torch.optim as optim

In [23]:
def save_loss_in_csv(model_num, loss):
    df = pd.read_csv('loss.csv')
    df = df.reindex(range(len(loss)))
    df[f"{model_num}"] = loss
    df.to_csv('loss.csv', index=False)

In [24]:
# build the gradient boosting regression model with PyTorch
class Net(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, dropout_rate):
        super(Net, self).__init__()
        self.dropout = nn.Dropout(p=dropout_rate)
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3= nn.Linear(hidden_size, hidden_size)
        self.fc4= nn.Linear(hidden_size, hidden_size)
        self.fc5= nn.Linear(hidden_size, output_size)
        self.relu = nn.ReLU()
        
        
    def forward(self, x):
        out = self.fc1(x)
        out = self.dropout(out)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.dropout(out)
        out = self.relu(out)
        out = self.fc3(out)
        out = self.dropout(out)
        out = self.relu(out)
        out = self.fc4(out)
        out = self.dropout(out)
        out = self.relu(out)
        out = self.fc5(out)
        return out
    
class CustomLoss(nn.Module):
    def __init__(self):
        super(CustomLoss, self).__init__()
        
    def forward(self, outputs, labels):
        # Define your custom loss function here
        # This is a simple example, replace it with your own function
        custom_loss = 3*(torch.abs(labels - outputs))*(torch.abs(labels - 1/3) + torch.abs(labels -2/3))
        return custom_loss
    

    
def objective(trial):
    # Sample hyperparameters
    input_size = 8
    hidden_size = trial.suggest_int('hidden_size', 3, 1000, log=True)  # Single hidden size for all layers
    output_size = 1
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-1, log=True)
    dropout_rate = trial.suggest_float('dropout_rate', 0.0, 0.3)

    # Instantiate the model with sampled hyperparameters
    model = Net(input_size, hidden_size, output_size, dropout_rate)

    # Define the loss function and optimizer
    criterion = CustomLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Training loop
    for epoch in range(100):
        training_loss = 0.0
        eval_loss = 0.0

        # Iterate over the DataLoader for training data
        for i, data in enumerate(train_loader):
            inputs, labels = data
            inputs = inputs.float()
            labels = labels.float()
            optimizer.zero_grad()
            outputs = model(inputs)
            outputs = outputs.reshape(-1)
            loss = criterion(outputs, labels).sum()
            loss.backward()
            optimizer.step()
            training_loss += loss.item()

        model.eval()
        outloss = 0
        # Iterate over the DataLoader for test data
        with torch.no_grad():
            for i, data in enumerate(test_loader):
                inputs, labels = data
                inputs = inputs.float()
                labels = labels.float()
                outputs = model(inputs).reshape(-1)
                loss = criterion(outputs, labels).sum()
                eval_loss += loss

        # Optuna logs the running loss for each epoch
        trial.report(eval_loss, epoch)

        # Handle pruning based on the intermediate value
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

    # Return the running loss as the objective value to minimize
    return eval_loss


class station_model():
    def __init__(self, station):
        self.station = station
        self.epoch_cnt = 40
        self.trial_cnt = 20

    def get_data_loader(self):
        df = pd.read_parquet(f'parquets/{self.station}train.parquet')
        TOT = df['tot'].iloc[0]
        df['sbi'] = df['sbi']/TOT
        df['time'] = df['time']/1440

        # x is dataset without 'sbi', y is 'sbi'
        X = df.drop(['tot', 'sbi','bemp' ,'act', 'tot', 'station'], axis=1)
        y = df['sbi']

        X = X.to_numpy()
        y = y.to_numpy()

        X = torch.from_numpy(X)
        y = torch.from_numpy(y)
        dataset = TensorDataset(X, y)

        # get train, test loader
        self.all_data_loader = torch.utils.data.DataLoader(dataset, batch_size=128, shuffle=True)

        # split train, test
        train_size = int(0.8 * len(dataset))
        test_size = len(dataset) - train_size
        train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

        # get train, test loader
        self.train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True)
        self.test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=128, shuffle=True)
        print(f'get data loader for {self.station}')

    def objective(self,trial):
        # Sample hyperparameters
        input_size = 8
        hidden_size = trial.suggest_int('hidden_size', 3, 2000, log=True)  # Single hidden size for all layers
        output_size = 1
        learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-1, log=True)
        dropout_rate = trial.suggest_float('dropout_rate', 0.2, 0.5)

        # Instantiate the model with sampled hyperparameters
        model = Net(input_size, hidden_size, output_size, dropout_rate)

        # Define the loss function and optimizer
        criterion = CustomLoss()
        optimizer = optim.Adam(model.parameters(), lr=learning_rate)

        # Training loop
        for epoch in range(self.epoch_cnt):
            training_loss = 0.0
            eval_loss = 0.0
            # Iterate over the DataLoader for training data
            for i, data in enumerate(self.train_loader):
                inputs, labels = data
                inputs = inputs.float()
                labels = labels.float()
                optimizer.zero_grad()
                outputs = model(inputs)
                outputs = outputs.reshape(-1)
                loss = criterion(outputs, labels).sum()
                loss.backward()
                optimizer.step()
                training_loss += loss.item()

            model.eval()
            # Iterate over the DataLoader for test data
            with torch.no_grad():
                for i, data in enumerate(self.test_loader):
                    inputs, labels = data
                    inputs = inputs.float()
                    labels = labels.float()
                    outputs = model(inputs).reshape(-1)
                    loss = criterion(outputs, labels).sum()
                    eval_loss += loss

            # Optuna logs the running loss for each epoch
            trial.report(eval_loss, epoch)
            # Handle pruning based on the intermediate value
            if trial.should_prune():
                raise optuna.exceptions.TrialPruned()
        # Return the running loss as the objective value to minimize
        return eval_loss      


    def get_best_param(self):
        # Create an Optuna Study
        study = optuna.create_study(direction='minimize', storage='sqlite:///db.sqlite3', study_name=f'{self.station}', load_if_exists=True)
        num_trials = len(study.trials)

        if num_trials < self.trial_cnt:
            study.optimize(self.objective, n_trials=self.trial_cnt)
        # Run the optimization process

        print(f'{self.station} eval_loss: {study.best_trial.value}')

        # Access the best hyperparameters
        best_params = study.best_trial.params

        self.best_params = best_params

        print(self.station,' Best hyperparameters:', best_params)

    def train(self):
        # Instantiate the final model with the best hyperparameters
        final_model = Net(8, self.best_params['hidden_size'], 1, self.best_params['dropout_rate'])

        # ... rest of your training code for the final model
        criterion = CustomLoss()
        optimizer = optim.Adam(final_model.parameters(), lr=self.best_params['learning_rate'])

        # train with whole dataset
        running_loss = 0.0
        loss_list = []
        for epoch in range(self.epoch_cnt):
            for i, data in enumerate(self.all_data_loader):
                inputs, labels = data
                inputs = inputs.float()
                labels = labels.float()
                optimizer.zero_grad()
                outputs = final_model(inputs)
                outputs = outputs.squeeze()
                loss = criterion(outputs, labels).sum()
                loss.backward()
                optimizer.step()
                running_loss += loss.item()/128
            print(f'epoch: {epoch+1}, loss: {running_loss/self.all_data_loader.__len__()}')
            loss_list.append(running_loss/self.all_data_loader.__len__())
            running_loss = 0.0
        save_loss_in_csv(self.station, loss_list)
        
        final_model.eval()
        # save model
        torch.save(final_model, f'models/{self.station}')

In [25]:
available_station = np.loadtxt('html.2023.final.data/sno_test_set.txt', dtype='str')
# available_station = available_station[:35]
folder_path = 'models/'  # replace with your actual folder path
trained_models = os.listdir(folder_path)
available_station = available_station[range(0, 100, 10)]    
available_station = np.setdiff1d(available_station, trained_models)
print(available_station)

['500101035' '500101093' '500101185' '500105066' '500119049' '500119059'
 '500119069']


In [26]:
for station_id in available_station:
    station = station_model(station_id)
    station.get_data_loader()
    station.get_best_param()
    station.train()
    print(f'{station_id} done')

[I 2023-12-26 17:32:26,419] Using an existing study with name '500101035' instead of creating a new one.


get data loader for 500101035


[I 2023-12-26 17:50:57,589] Trial 4 finished with value: 4585.388671875 and parameters: {'hidden_size': 1790, 'learning_rate': 0.03463508502787367, 'dropout_rate': 0.20666852054502455}. Best is trial 2 with value: 2843.518310546875.
[I 2023-12-26 17:51:21,187] Trial 5 finished with value: 4509.77783203125 and parameters: {'hidden_size': 3, 'learning_rate': 0.0028522938552576565, 'dropout_rate': 0.35512273423972884}. Best is trial 2 with value: 2843.518310546875.
[I 2023-12-26 17:51:22,644] Trial 6 pruned. 
[I 2023-12-26 17:51:46,807] Trial 7 finished with value: 2910.866455078125 and parameters: {'hidden_size': 42, 'learning_rate': 0.012223742679691054, 'dropout_rate': 0.3128331746866366}. Best is trial 2 with value: 2843.518310546875.
[I 2023-12-26 17:52:56,153] Trial 8 finished with value: 2741.30712890625 and parameters: {'hidden_size': 242, 'learning_rate': 0.0002175697336657348, 'dropout_rate': 0.3345309177813923}. Best is trial 8 with value: 2741.30712890625.
[I 2023-12-26 17:52:

500101035 eval_loss: 2717.1982421875
500101035  Best hyperparameters: {'hidden_size': 492, 'learning_rate': 0.0009311824173693834, 'dropout_rate': 0.41483751855165996}
epoch: 1, loss: 0.2740969231927582
epoch: 2, loss: 0.22809065771768647
epoch: 3, loss: 0.21936198044405766
epoch: 4, loss: 0.2141165563781432
epoch: 5, loss: 0.21121932394529513
epoch: 6, loss: 0.20951932915738325
epoch: 7, loss: 0.2079972903557889
epoch: 8, loss: 0.20729470674280095
epoch: 9, loss: 0.2060112682468604
epoch: 10, loss: 0.20474451079470532
epoch: 11, loss: 0.20537590629455307
epoch: 12, loss: 0.20532331395939799
epoch: 13, loss: 0.20400357847126366
epoch: 14, loss: 0.20378213665768322
epoch: 15, loss: 0.20390419147089514
epoch: 16, loss: 0.20365609382043542
epoch: 17, loss: 0.20348543230359675
epoch: 18, loss: 0.2032204017069119
epoch: 19, loss: 0.2028490648479778
epoch: 20, loss: 0.20306813441229948
epoch: 21, loss: 0.20273200457632853
epoch: 22, loss: 0.20177114007256627
epoch: 23, loss: 0.20217137907292

[I 2023-12-26 18:17:21,597] A new study created in RDB with name: 500101093


...

500119059 eval_loss: 2787.47314453125
500119059  Best hyperparameters: {'hidden_size': 52, 'learning_rate': 1.4426729513064302e-05, 'dropout_rate': 0.44865381760510564}
epoch: 1, loss: 0.3038349588243957
epoch: 2, loss: 0.2715599414149712
epoch: 3, loss: 0.2586920060880521
epoch: 4, loss: 0.24863527130082017
epoch: 5, loss: 0.24001203542396451
epoch: 6, loss: 0.23319167080766035
epoch: 7, loss: 0.2271730578382602
epoch: 8, loss: 0.22224681152203113
epoch: 9, loss: 0.2181908471890145
epoch: 10, loss: 0.21479944341261767
epoch: 11, loss: 0.21143036690682016
epoch: 12, loss: 0.20855448514639188
epoch: 13, loss: 0.20576747824124225
epoch: 14, loss: 0.20320563918763015
epoch: 15, loss: 0.20080071873637811
epoch: 16, loss: 0.19857599023228956
epoch: 17, loss: 0.19651231090383797
epoch: 18, loss: 0.1948678814277807
epoch: 19, loss: 0.1937041381799842
epoch: 20, loss: 0.19312238188730277
epoch: 21, loss: 0.19288365252927128
epoch: 22, loss: 0.1927760601641828
epoch: 23, loss: 0.192698997372264

[I 2023-12-26 21:54:47,795] A new study created in RDB with name: 500119069


epoch: 40, loss: 0.1925825920560597
500119059 done
get data loader for 500119069


[I 2023-12-26 21:55:04,756] Trial 0 finished with value: 3561.4873046875 and parameters: {'hidden_size': 89, 'learning_rate': 0.011689462645289144, 'dropout_rate': 0.46263108707921474}. Best is trial 0 with value: 3561.4873046875.
[I 2023-12-26 21:55:18,548] Trial 1 finished with value: 3495.05029296875 and parameters: {'hidden_size': 15, 'learning_rate': 0.00035194664380895754, 'dropout_rate': 0.4464779931275333}. Best is trial 1 with value: 3495.05029296875.
[I 2023-12-26 21:55:37,583] Trial 2 finished with value: 3043.6611328125 and parameters: {'hidden_size': 112, 'learning_rate': 0.00032724523442694426, 'dropout_rate': 0.23501656413477892}. Best is trial 2 with value: 3043.6611328125.
[I 2023-12-26 21:55:50,936] Trial 3 finished with value: 8668.1396484375 and parameters: {'hidden_size': 5, 'learning_rate': 1.4960488834924383e-05, 'dropout_rate': 0.21033097123646827}. Best is trial 2 with value: 3043.6611328125.
[I 2023-12-26 21:56:04,247] Trial 4 finished with value: 4744.9013671

500119069 eval_loss: 2984.7578125
500119069  Best hyperparameters: {'hidden_size': 40, 'learning_rate': 0.0007622312276149493, 'dropout_rate': 0.24005496758210995}
epoch: 1, loss: 0.5901406243001396
epoch: 2, loss: 0.3732312958017902
epoch: 3, loss: 0.30701625458961174
epoch: 4, loss: 0.29122329512399736
epoch: 5, loss: 0.2859581648575282
epoch: 6, loss: 0.28360406159834506
epoch: 7, loss: 0.2810295885249998
epoch: 8, loss: 0.2792215577698919
epoch: 9, loss: 0.27695947516234964
epoch: 10, loss: 0.27457013125506996
epoch: 11, loss: 0.27283629052093933
epoch: 12, loss: 0.26886162700141286
epoch: 13, loss: 0.266990836456184
epoch: 14, loss: 0.2646002691713719
epoch: 15, loss: 0.26088114857777667
epoch: 16, loss: 0.25931775694742254
epoch: 17, loss: 0.25738622471820205
epoch: 18, loss: 0.25520056543341896
epoch: 19, loss: 0.25501558301648547
epoch: 20, loss: 0.2532317280821359
epoch: 21, loss: 0.2545983365380951
epoch: 22, loss: 0.2525601695709412
epoch: 23, loss: 0.25487227137176155
epoch

In [27]:
# read loss.csv 
# train those station with loss avg > 0.35
loss_df = pd.read_csv('loss.csv')
# get station with loss avg > 0.35
station_list = []
for col in loss_df.columns:
    # if last element > 0.35, train this station
    if loss_df[col].iloc[-1] > 0.35:
        col = float(col)
        col = int(col)
        station_list.append(col)
print(station_list)

# train those station
for station_id in station_list:
    station = station_model(station_id)
    station.get_data_loader()
    station.get_best_param()
    station.train()
    print(f'train {station} done!')

[I 2023-12-26 22:15:41,357] Using an existing study with name '500101185' instead of creating a new one.


[500101185, 500105066]
get data loader for 500101185
500101185 eval_loss: 5286.84228515625
500101185  Best hyperparameters: {'hidden_size': 356, 'learning_rate': 0.0007141888333661958, 'dropout_rate': 0.29781193900891495}
epoch: 1, loss: 0.49009636689613506
epoch: 2, loss: 0.4264851840673735
epoch: 3, loss: 0.40807564536154994
epoch: 4, loss: 0.3988940385283857
epoch: 5, loss: 0.3935745941874172
epoch: 6, loss: 0.3898067466252668
epoch: 7, loss: 0.387160284473895
epoch: 8, loss: 0.38495025503406954
epoch: 9, loss: 0.3824755400759963
epoch: 10, loss: 0.38162313460434044
epoch: 11, loss: 0.3806985300099185
epoch: 12, loss: 0.37921132009398545
epoch: 13, loss: 0.3788986421005236
epoch: 14, loss: 0.3792520542629063
epoch: 15, loss: 0.3771976936602063
epoch: 16, loss: 0.3771499465381928
epoch: 17, loss: 0.377403168512333
epoch: 18, loss: 0.3766448287072653
epoch: 19, loss: 0.37613593387167626
epoch: 20, loss: 0.3776716471581561
epoch: 21, loss: 0.37592414139610963
epoch: 22, loss: 0.3771107

[I 2023-12-26 22:16:42,487] Using an existing study with name '500105066' instead of creating a new one.


epoch: 40, loss: 0.37378710716677255
train <__main__.station_model object at 0x168e6ea90> done!
get data loader for 500105066
500105066 eval_loss: 5370.21337890625
500105066  Best hyperparameters: {'hidden_size': 224, 'learning_rate': 0.00022976864738344293, 'dropout_rate': 0.20056603384926994}
epoch: 1, loss: 0.43624664471203534
epoch: 2, loss: 0.403627388244316
epoch: 3, loss: 0.3988242933159307
epoch: 4, loss: 0.39586047071436936
epoch: 5, loss: 0.39096930966327326
epoch: 6, loss: 0.39025656943962006
epoch: 7, loss: 0.3876387327129304
epoch: 8, loss: 0.38575300579503774
epoch: 9, loss: 0.3845306245860421
epoch: 10, loss: 0.38363710893594366
epoch: 11, loss: 0.383116187797583
epoch: 12, loss: 0.38275671957050944
epoch: 13, loss: 0.3817312025690994
epoch: 14, loss: 0.3801361272992472
epoch: 15, loss: 0.3795787002923809
epoch: 16, loss: 0.3797081894067362
epoch: 17, loss: 0.3787905340835478
epoch: 18, loss: 0.37868835050190097
epoch: 19, loss: 0.37846252735684677
epoch: 20, loss: 0.377

In [30]:
# get Eout for each station


IsADirectoryError: [Errno 21] Is a directory: 'models/'

In [28]:
df = pd.read_csv('loss.csv')
df

,1,500101001,500101013,500101025,500101035,500101093,500101185,500105066,500119049,500119059,500119069
0,NaN,0.358454,0.496978,0.238550,0.274097,0.287640,0.490096,0.436247,0.252419,0.303835,0.590141
1,NaN,0.308635,0.473846,0.181518,0.228091,0.273880,0.426485,0.403627,0.212392,0.271560,0.373231
2,NaN,0.300904,0.454955,0.174322,0.219362,0.269760,0.408076,0.398824,0.205252,0.258692,0.307016
3,NaN,0.297149,0.441614,0.170558,0.214117,0.267881,0.398894,0.395860,0.202949,0.248635,0.291223
4,NaN,0.294615,0.432606,0.168464,0.211219,0.266337,0.393575,0.390969,0.200847,0.240012,0.285958
5,NaN,0.293744,0.429598,0.166457,0.209519,0.265453,0.389807,0.390257,0.200325,0.233192,0.283604
6,NaN,0.292254,0.426047,0.165044,0.207997,0.264652,0.387160,0.387639,0.199145,0.227173,0.281030
7,NaN,0.290828,0.424440,0.164444,0.207295,0.263007,0.384950,0.385753,0.198478,0.222247,0.279222
8,NaN,0.291058,0.421728,0.163466,0.206011,0.263070,0.382476,0.384531,0.197904,0.218191,0.276959
9,NaN,0.290285,0.420428,0.162087,0.204745,0.262026,0.381623,0.383637,0.197369,0.214799,0.274570
